In [3]:
#Importing libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [4]:
#Data reaing an parsing
r = requests.get('https://forecast.weather.gov/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996')
soup = BeautifulSoup(r.text,'html.parser')


#data collection and dataframe creation
forecast = soup.find_all('div', attrs={'id':'detailed-forecast-body'})

c = -1
f = []
for i in forecast[0].findAll('div'):
    c += 1
    if c%3 == 0:
        continue
    f.append(i.text)

df = pd.DataFrame([f[k:k+2] for k in range(0, len(f), 2)], columns=['Day', 'Forecast'])

df['near'] = df.Forecast.apply(lambda x: re.findall(r'(?<=\bnear\s)(\w+)', x))
df['around'] = df.Forecast.apply(lambda x: re.findall(r'(?<=\baround\s)(\w+)', x))
df['temp'] = df['near'] + df['around']
df['temp'] = df.temp.apply(lambda x: x[0]+'° F')
df = df.drop(['near', 'around'], axis=1)
df

,Day,Forecast,temp
0,Tonight,"Mostly clear, with a low around 54. West wind ...",54° F
1,Friday,"Mostly sunny, with a high near 66. Light west ...",66° F
2,Friday Night,"Partly cloudy, with a low around 56. West sout...",56° F
3,Saturday,"Partly sunny, with a high near 68. Southwest w...",68° F
4,Saturday Night,"Mostly cloudy, with a low around 55. Southwest...",55° F
5,Sunday,"Partly sunny, with a high near 68.",68° F
6,Sunday Night,"Mostly clear, with a low around 53.",53° F
7,Monday,"Sunny, with a high near 72.",72° F
8,Monday Night,"Mostly clear, with a low around 53.",53° F
9,Tuesday,"Sunny, with a high near 73.",73° F


In [5]:
df.to_csv('USA-weather.csv',index=False,encoding='utf-8')